In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize # cắt câu thành từng từ (tokenize) bằng thư viện NLTK (Natural Language Toolkit).
from nltk.corpus import stopwords       # danh sách các từ không có nhiều ý nghĩa (như “và”, “là”, “của”,...) để loại bỏ khi xử lý ngôn ngữ.


In [3]:
# Đường dẫn đến file CSV và folder ảnh
csv_path = '/kaggle/input/okok1230/dev_images_json_csv.csv'  
image_folder = '/kaggle/input/2025-sum-dpl-302-m/devset_images/devset_images'
csv_path_test = '/kaggle/input/2025-sum-dpl-302-m/test.csv'
image_folder_test = '/kaggle/input/2025-sum-dpl-302-m/testset_images/testset_images'

# Đọc file CSV
df = pd.read_csv(csv_path)
df_test = pd.read_csv(csv_path_test)

In [4]:
# Xử lý dữ liệu văn bản: Gộp các cột title, description, user_tag
import re #Dùng để tìm kiếm, kiểm tra, và xử lý chuỗi văn bản theo mẫu (pattern)

def preprocess_text(text):
    if pd.isna(text):  # Xử lý NaN
        return ""
    
    text = str(text).lower() # Ép text thành chuỗi (trường hợp nó không phải chuỗi), sau đó chuyển toàn bộ thành chữ thường để chuẩn hóa.
    text = re.sub(r'[^\w\s]', '', text)  # Xóa ký tự đặc biệt # [^\w\s] nghĩa là "mọi ký tự không phải chữ, số, gạch dưới, hoặc khoảng trắng"
    tokens = word_tokenize(text) #word_tokenize từ thư viện NLTK để chia câu thành từng từ riêng biệt (token).
    
    stop_words = set(stopwords.words('english')) #Lấy tập các từ stopwords tiếng Anh
    important_words = {'no', 'not', 'nor', 'never'} # Tạo tập từ quan trọng (đảo nghĩa) dù chúng là stopwords nhưng cần giữ lại để không làm mất nghĩa câu.
    filtered_tokens = [t for t in tokens if (t not in stop_words or t in important_words)]
    
    return ' '.join(filtered_tokens) #Ghép các từ đã lọc thành chuỗi mới, cách nhau bằng dấu cách, rồi trả về kết quả.


In [5]:
# Tạo cột combined_text
df['combined_text'] = df['title'].apply(preprocess_text) + ' ' + \
                      df['description'].apply(preprocess_text) + ' ' + \
                      df['user_tag'].apply(preprocess_text)

df_test['combined_text'] = df_test['title'].apply(preprocess_text) + ' ' + \
                           df_test['description'].apply(preprocess_text) + ' ' + \
                           df_test['user_tags'].apply(preprocess_text)

In [6]:
from transformers import BertTokenizer #bộ tokenizer chuẩn để chuyển câu thành token cho mô hình BERT.
import torchvision.transforms as T 

# Load tokenizer BERT base uncased
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # Tạo đối tượng tokenizer cho BERT base uncased (không phân biệt hoa thường), 
                                                               # from_pretrained tải sẵn vocab và cấu hình của tokenizer từ thư viện Hugging Face.
                                                               # Ví dụ: câu "I love AI!" có thể được tách thành các token: ["I", "love", "AI", "!"].


# Image transform (ImageNet chuẩn)
image_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(), # Chuyển ảnh từ dạng PIL Image hoặc numpy array sang tensor PyTorch.  pixel mới = pixel cũ / 255
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #Mô hình CNN như ResNet, VGG... được huấn luyện với ảnh có phân phối màu sắc trung bình và độ lệch chuẩn như trên.
])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

def find_image_path(folder, image_id):
    for ext in ['png', 'jpg', 'jpeg', 'gif']:
        path = os.path.join(folder, f"{image_id}.{ext}")
        if os.path.exists(path):
            return path
    return None

In [8]:
class FloodDataset(Dataset):
    def __init__(self, dataframe, image_folder, tokenizer, max_length=128, transform=None):
        self.data = dataframe.reset_index(drop=True) #chứa thông tin ảnh và văn bản (image_id, combined_text, label)
        self.image_folder = image_folder #thư mục chứa ảnh
        self.tokenizer = tokenizer #BERT tokenizer để mã hóa văn bản
        self.max_length = max_length 
        self.transform = transform if transform else transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx] #trả về một Series (tương tự một dòng) chứa thông tin của ảnh và văn bản tương ứng ở vị trí idx.
        image_id = str(row['image_id']) #Lấy giá trị trong cột 'image_id' của dòng đó và chuyển thành kiểu chuỗi (string).
        
        image_path = find_image_path(self.image_folder, image_id) #Gọi hàm find_image_path để tìm đường dẫn file ảnh trong thư mục self.image_folder dựa vào image_id
        if image_path is None:
            print(f"Image not found for id {image_id}")
            image = Image.new('RGB', (224, 224), (0, 0, 0)) # tạo ảnh đen
        else:
            image = Image.open(image_path).convert('RGB') # định dang sang RGB
        
        image = self.transform(image)
        text = row['combined_text']
        
        encoded = self.tokenizer(
            text,
            max_length=self.max_length,
            truncation=True, #nếu văn bản dài hơn max_length thì cắt bớt cho vừa
            padding='max_length',# nếu văn bản ngắn hơn max_length thì thêm padding cho đủ độ dài.
            return_tensors='pt' #trả về kết quả dưới dạng tensor PyTorch.
        )
        
        label = torch.tensor(row['label'], dtype=torch.float) if 'label' in row else torch.tensor(0.0)
        
        return {
            'image': image,
            'input_ids': encoded['input_ids'].squeeze(0),
            'attention_mask': encoded['attention_mask'].squeeze(0),
            'label': label
        }

In [9]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

train_dataset = FloodDataset(df, image_folder, tokenizer, transform=image_transform)
test_dataset = FloodDataset(df_test, image_folder_test, tokenizer, transform=image_transform)
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.01)

train_loader = DataLoader(train_dataset, batch_size=80, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=80, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=80, shuffle=False, num_workers=2)



In [10]:
import torch
import torch.nn as nn
from transformers import BertModel, ViTModel

class FloodModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.vit = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k") #Tải mô hình ViT đã được huấn luyện trước.
        self.vit_fc = nn.Sequential(
            nn.Linear(768, 256), ## Giảm chiều từ 768 → 256
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3)
        )

        self.text_model = BertModel.from_pretrained("bert-base-uncased") #Tải mô hình BERT đã được huấn luyện.
        self.text_fc = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3)
        )

        self.classifier = nn.Sequential(
            nn.Linear(512, 128),  # Ghép 256 (ảnh) + 256 (text) = 512 → 128
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            # nn.Sigmoid()  
        )

    def forward(self, image, input_ids, attention_mask):
        img_feat = self.vit(pixel_values=image).last_hidden_state[:, 0, :] #[:, 0, :]: chỉ lấy CLS token (tức token tóm tắt toàn bộ ảnh) → shape (batch_size, 768)
        img_feat = self.vit_fc(img_feat)

        text_feat = self.text_model(input_ids=input_ids, attention_mask=attention_mask).pooler_output #.pooler_output: đầu ra của token [CLS] (biểu diễn toàn câu) → shape (batch_size, 768)
        text_feat = self.text_fc(text_feat)

        combined = torch.cat((img_feat, text_feat), dim=1)
        return self.classifier(combined) #ghép theo chiều đặc trưng → kết quả có shape (batch_size, 512)

2025-07-01 13:43:05.121116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751377385.632260      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751377385.746628      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = FloodModel()
model = nn.DataParallel(model)  # Chạy trên nhiều GPU
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [12]:
# Vòng lặp huấn luyện và đánh giá
for epoch in range(5):
    # Huấn luyện
    model.train()
    total_train_loss = 0
    train_correct = 0
    train_total = 0

    for batch in train_loader:
        images = batch['image'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device).float().unsqueeze(1)
    
        outputs = model.forward(images, input_ids, attention_mask)  
        loss = criterion(outputs, labels)                      
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        total_train_loss += loss.item()
        predictions = (outputs > 0.5).float()
        train_correct += (predictions == labels).sum().item()
        train_total += labels.size(0)
    avg_train_loss = total_train_loss / len(train_loader)
    train_accuracy = train_correct / train_total

    # Đánh giá trên validation
    model.eval()
    total_val_loss = 0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for batch in val_loader:
            images = batch['image'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device).float().unsqueeze(1)
    
            outputs = model.forward(images, input_ids, attention_mask)
            loss = criterion(outputs, labels)
    
            total_val_loss += loss.item()
            predictions = (outputs > 0.5).float()
            val_correct += (predictions == labels).sum().item()
            val_total += labels.size(0)

    
    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    # In kết quả
    print(f"Epoch {epoch+1} -> Train Loss: {avg_train_loss:.4f} - Train Accuracy: {train_accuracy:.4f} || "
          f"Val Loss: {avg_val_loss:.4f} - Val Accuracy: {val_accuracy:.4f}")

Epoch 1 -> Train Loss: 0.2761 - Train Accuracy: 0.9047 || Val Loss: 0.4353 - Val Accuracy: 0.8679
Epoch 2 -> Train Loss: 0.1270 - Train Accuracy: 0.9644 || Val Loss: 0.5257 - Val Accuracy: 0.8491
Epoch 3 -> Train Loss: 0.0736 - Train Accuracy: 0.9835 || Val Loss: 0.4019 - Val Accuracy: 0.8868
Epoch 4 -> Train Loss: 0.0516 - Train Accuracy: 0.9901 || Val Loss: 0.4619 - Val Accuracy: 0.8868
Epoch 5 -> Train Loss: 0.0330 - Train Accuracy: 0.9943 || Val Loss: 0.2931 - Val Accuracy: 0.9245


In [13]:
import torch
import pandas as pd

model.eval() #Đây là một phương thức (method) của lớp mô hình trong PyTorch, dùng để chuyển mô hình sang chế độ đánh giá (evaluation mode).
preds = []
image_ids = []

with torch.no_grad():
    for batch in test_loader:
        images = batch['image'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model.forward(images, input_ids, attention_mask)
        preds_batch = (outputs > 0.5).long().cpu().numpy().squeeze() 

        preds.extend(preds_batch) #.extend() sẽ thêm từng phần tử của batch_ids vào image_ids
 
        batch_ids = batch['image_id'] if 'image_id' in batch else None
        if batch_ids is not None:
            image_ids.extend(batch_ids)
        else:
            start_idx = len(image_ids)
            end_idx = start_idx + len(preds_batch)
            image_ids.extend(df_test['image_id'].iloc[start_idx:end_idx].tolist()) # chuyển thành list

In [ ]:
df_pred = pd.DataFrame({
    'id': image_ids,
    'label': preds
})

df_pred.to_csv('test_predictions.csv', index=False)
print("Saved test predictions to test_predictions.csv")